In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='/kaggle/working/mnist-data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='/kaggle/working/mnist-data', train=False, transform=transforms.ToTensor(), download=True)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)

100%|██████████| 9.91M/9.91M [00:10<00:00, 919kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 61.4kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 569kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.52MB/s]


In [2]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        
        # encoder part
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        # decoder part
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
        
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h) # mu, log_var
    
    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h)) 
    
    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 784))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()


In [3]:
for i,(data,_) in enumerate(train_loader):
    print(data.shape)

    h = nn.Conv2d(1, 16, 3, 2, 1)(data)
    print(h.shape)
    h = nn.Conv2d(16, 32, 3, 2, 1)(h)
    print(h.shape)
    h = nn.Linear(7*7*32, 2)(h.view(-1, 7*7*32))
    print(h.shape)

    h = nn.Linear(2, 7*7*32)(h).view(-1, 32, 7, 7)
    print(h.shape)
    h = nn.ConvTranspose2d(32, 16, 3, 2, output_padding=1, padding=1)(h)
    print(h.shape)
    h = nn.ConvTranspose2d(16, 1, 3, 2, output_padding=1, padding=1)(h)
    print(h.shape)

    break

torch.Size([100, 1, 28, 28])
torch.Size([100, 16, 14, 14])
torch.Size([100, 32, 7, 7])
torch.Size([100, 2])
torch.Size([100, 32, 7, 7])
torch.Size([100, 16, 14, 14])
torch.Size([100, 1, 28, 28])


In [4]:
class VAE_CNN(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE_CNN, self).__init__()
        
        # encoder part
        # self.fc1 = nn.Linear(x_dim, h_dim1)
        # self.fc2 = nn.Linear(h_dim1, h_dim2)
        # self.fc31 = nn.Linear(h_dim2, z_dim)
        # self.fc32 = nn.Linear(h_dim2, z_dim)
        self.cnn1 = nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1)
        self.cnn2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.fc3 = nn.Linear(32*7*7, h_dim1)
        self.fc4 = nn.Linear(h_dim1, h_dim2)
        self.fc51 = nn.Linear(h_dim2, z_dim)
        self.fc52 = nn.Linear(h_dim2, z_dim)
 # decoder part
        # self.fc4 = nn.Linear(z_dim, h_dim2)
        # self.fc5 = nn.Linear(h_dim2, h_dim1)
        # self.fc6 = nn.Linear(h_dim1, x_dim)
        self.fc6 = nn.Linear(z_dim, h_dim2)
        self.fc7 = nn.Linear(h_dim2, h_dim1)
        self.fc8 = nn.Linear(h_dim1, 32*7*7)
        self.tcnn9 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, output_padding=1, padding=1)
        self.tcnn10 = nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, output_padding=1, padding=1)
        
    def encoder(self, x):
        # h = F.relu(self.fc1(x))
        # h = F.relu(self.fc2(h))
        # return self.fc31(h), self.fc32(h) # mu, log_var
        h = F.relu(self.cnn1(x))
        h = F.relu(self.cnn2(h))
        h = F.relu(self.fc3(h.view(-1, 32*7*7)))
        h = F.relu(self.fc4(h))
        return self.fc51(h), self.fc52(h)

    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        # h = F.relu(self.fc4(z))
        # h = F.relu(self.fc5(h))
        # return F.sigmoid(self.fc6(h)) 
        h = F.relu(self.fc6(z))
        h = F.relu(self.fc7(h))
        h = F.relu(self.fc8(h)).view(-1, 32, 7, 7)
        h = F.relu(self.tcnn9(h))
        return torch.sigmoid(self.tcnn10(h))
    
    def forward(self, x):
        # mu, log_var = self.encoder(x.view(-1, 784))
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae = VAE_CNN(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [5]:
class VAE_CNN_Singular(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE_CNN_Singular, self).__init__()
        
        # encoder part
        self.cnn1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        #self.cnn2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.fc3 = nn.Linear(32*14*14, h_dim1)
        self.fc4 = nn.Linear(h_dim1, h_dim2)
        self.fc51 = nn.Linear(h_dim2, z_dim)
        self.fc52 = nn.Linear(h_dim2, z_dim)

        # decoder part
        self.fc6 = nn.Linear(z_dim, h_dim2)
        self.fc7 = nn.Linear(h_dim2, h_dim1)
        self.fc8 = nn.Linear(h_dim1, 32*14*14)
        #self.tcnn9 = nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, output_padding=1, padding=1)
        self.tcnn10 = nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, output_padding=1, padding=1)
        
    def encoder(self, x):
        h = F.relu(self.cnn1(x))
        h = F.relu(self.cnn2(h))
        h = F.relu(self.fc3(h.view(-1, 32*7*7)))
        h = F.relu(self.fc4(h))
        return self.fc51(h), self.fc52(h)

    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu) # return z sample
        
    def decoder(self, z):
        h = F.relu(self.fc6(z))
        h = F.relu(self.fc7(h))
        h = F.relu(self.fc8(h)).view(-1, 32, 7, 7)
        h = F.relu(self.tcnn9(h))
        return torch.sigmoid(self.tcnn10(h))
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var

# build model
vae_singular = VAE_CNN_Singular(x_dim=784, h_dim1= 512, h_dim2=256, z_dim=2)
if torch.cuda.is_available():
    vae.cuda()

In [6]:
vae

VAE_CNN(
  (cnn1): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (cnn2): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc3): Linear(in_features=1568, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc51): Linear(in_features=256, out_features=2, bias=True)
  (fc52): Linear(in_features=256, out_features=2, bias=True)
  (fc6): Linear(in_features=2, out_features=256, bias=True)
  (fc7): Linear(in_features=256, out_features=512, bias=True)
  (fc8): Linear(in_features=512, out_features=1568, bias=True)
  (tcnn9): ConvTranspose2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  (tcnn10): ConvTranspose2d(16, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
)

In [7]:
vae_singular

VAE_CNN_Singular(
  (cnn1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (fc3): Linear(in_features=6272, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=256, bias=True)
  (fc51): Linear(in_features=256, out_features=2, bias=True)
  (fc52): Linear(in_features=256, out_features=2, bias=True)
  (fc6): Linear(in_features=2, out_features=256, bias=True)
  (fc7): Linear(in_features=256, out_features=512, bias=True)
  (fc8): Linear(in_features=512, out_features=6272, bias=True)
  (tcnn10): ConvTranspose2d(32, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
)

In [12]:
optimizer = optim.Adam(vae.parameters())
# return reconstruction error + KL divergence losses
def loss_function(recon_x, x, mu, log_var):
    # BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return BCE + KLD

In [13]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cpu()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [ ]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cpu()
            recon, mu, log_var = vae(data)
            
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [14]:
for epoch in range(1, 51):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 583.328242
Train Epoch: 1 [10000/60000 (17%)]	Loss: 198.451758
Train Epoch: 1 [20000/60000 (33%)]	Loss: 194.958887
Train Epoch: 1 [30000/60000 (50%)]	Loss: 179.611973
Train Epoch: 1 [40000/60000 (67%)]	Loss: 180.459688
Train Epoch: 1 [50000/60000 (83%)]	Loss: 187.943379
====> Epoch: 1 Average loss: 198.6540


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
with torch.no_grad():
    z = torch.randn(64, 2).cpu()
    for i in range(2):
        z[:, i] = torch.linspace(-5, 0, 64)

    sample = vae.decoder(z).cpu()
    
    save_image(sample.view(64, 1, 28, 28), '/kaggle/working/sample1_' + '.png')
    

None


In [19]:
print(sample.view(64, 1, 28, 28))

tensor([[[[2.2655e-27, 9.5233e-36, 1.4196e-33,  ..., 2.0901e-20,
           2.0386e-20, 1.6351e-18],
          [4.6950e-31, 0.0000e+00, 4.3078e-39,  ..., 9.6937e-23,
           1.8220e-19, 3.3764e-15],
          [1.3945e-29, 0.0000e+00, 0.0000e+00,  ..., 6.1022e-21,
           1.8666e-18, 1.3398e-13],
          ...,
          [8.3594e-30, 4.7067e-29, 6.9267e-24,  ..., 4.2606e-23,
           1.8737e-23, 1.0894e-19],
          [2.0808e-22, 4.8173e-31, 2.5587e-24,  ..., 1.3288e-27,
           1.9216e-21, 8.1219e-14],
          [2.6182e-19, 4.0590e-27, 1.8386e-17,  ..., 1.8349e-21,
           1.3749e-14, 6.2008e-14]]],


        [[[5.5300e-27, 3.0668e-35, 4.2671e-33,  ..., 4.0008e-20,
           3.9098e-20, 2.9518e-18],
          [1.2991e-30, 0.0000e+00, 1.5554e-38,  ..., 1.9833e-22,
           3.3731e-19, 5.3920e-15],
          [3.6732e-29, 0.0000e+00, 0.0000e+00,  ..., 1.1737e-20,
           3.3276e-18, 2.0368e-13],
          ...,
          [2.1619e-29, 1.1834e-28, 1.4755e-23,  ..., 9.06